In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parte 1: Cálculo vetorial

## Questão 1: Derivadas direcionais

As derivadas direcionais são obtidas por um limite um pouco mais complicado:

$$ \frac{\partial f}{\partial v}(x)= \lim_{h\to0} \frac{f(x+hv) - f(x)}{h}. $$

(às vezes, também se escreve $\nabla_v f(x)$ ou $f'_v(x)$ para a derivada direcional.)

Generalize a função `df` para que ela calcule derivadas direcionais.

In [ ]:
def df(f,x,v,h=1e-8):
    # YOUR CODE HERE
    raise NotImplementedError()

### Algumas funções vetoriais

In [ ]:
def norm1(x):
    return np.sum(np.abs(x))
def norm2(x):
    return np.sum(x**2)
def estranha(x):
    x1,x2 = x
    return np.cos(x1) + 2*np.sin(x2/2)

#### Testes simples

In [ ]:
assert np.isclose(df(norm2, np.array([3,4]), np.array([0,2])), 16)
assert np.isclose(df(norm2, np.array([3,4]), np.array([1,-1])), -2)

In [ ]:
assert np.isclose(df(estranha, np.array([1,2]), np.array([2,1])), -1.1426397161784507)
assert np.isclose(df(estranha, np.array([1,2]), np.array([2,1]), h=1e-3), -1.1426397161784507, rtol=2e-3)

In [ ]:
assert np.isclose(df(norm1, np.array([3,3]), np.array([0,2])), 2)
assert np.isclose(df(norm1, np.array([3,3]), np.array([1,-1])), 0)

#### Testando propriedades

In [ ]:
assert np.isclose(df(norm2, np.array([0,3]), np.array([1,0])), 
                  -df(norm2, np.array([0,3]), np.array([-1,0])))

assert np.isclose(df(norm1, np.array([0,3]), np.array([1,0])), 
                  df(norm1, np.array([0,3]), np.array([-1,0])))

**Pergunta:** Como interpretar (em Cálculo) estas duas últimas igualdades?  Porque a segunda não tem um sinal de menos?

YOUR ANSWER HERE

## Questão 2: Gradientes

Vamos usar a nova função `df` para calcular [gradientes](https://pt.wikipedia.org/wiki/Gradiente) e outros objetos do cálculo vetorial.

Usando a função `len` (para descobrir a dimensão!), implemente `grad(f,x,delta)`,
onde cada derivada parcial é calculada com um passo de tamanho $\delta$.

In [ ]:
def grad(f,x,delta=1e-8):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
p = np.array([3,4])
assert np.allclose(grad(norm2, p, delta=1e-5), 2*p, rtol=1e-5)

In [ ]:
assert np.allclose(grad(norm1, np.array([3,4])), [1,1])
assert np.allclose(grad(norm1, np.array([3,-4])), [1,-1])

In [ ]:
ans = [-0.14112000805986724, -0.41614683654714246]
assert np.allclose(grad(estranha, np.array([3,4]), 1e-8), ans)

## Questão 3: Funções vetoriais

Agora, vejamos o que acontece quando a função $f$ vai de $R^n$ em $R^m$.
Supondo que a função (programada) `f` receba um vetor (de dimensão $n$) e retorne um vetor (de dimensão $m$),
dê a fórmula da $j$-ésima coordenada do vetor `df(f,x,v,h)`.

**Sugestão:** use $f_j(p)$ para a $j$-ésima coordenada do vetor `f(p)`.

YOUR ANSWER HERE

### Mais funções vetoriais

In [ ]:
def curva1(t):
    return np.array([np.cos(t), np.sin(t), t])

def superficie1(t):
    u,v = t
    return np.array([u*np.exp(v-u), v*np.cos(u+v), np.sin(v)])

In [ ]:
assert np.allclose(df(superficie1, np.array([0,0]), np.array([1,2])), [1,2,2])

In [ ]:
ans = [-0.9092974268256819, -0.41614683654714246, 1.0]
assert np.allclose(df(curva1, 2, 1, 1e-5), ans, rtol=2e-5)

## Questão 4: Ordem dos eixos

A sua função `grad` deveria retornar um `np.array`, e para a função `superficie1`,
de $R^2$ em $R^3$, isso deve dar a matriz com todas as derivadas parciais.

In [ ]:
grad(superficie1, np.array([1,2]), delta=1e-5)

Observando o resultado acima, o que você pode dizer sobre as linhas e colunas da matriz gradiente?
Elas coincidem com a ordem típica do cálculo 2?

YOUR ANSWER HERE

## Questão 5: Divergente

Adapte o cálculo do gradiente para obter o divergente de uma função vetorial de $R^n$ em $R^n$:

$$ \text{div} F(x) = \sum \frac{\partial f}{\partial x_i}(x). $$

In [ ]:
def div(f,x,delta=1e-8):
    # YOUR CODE HERE
    raise NotImplementedError()

### Mais funções ainda!

In [ ]:
def polar(p):
    rho,theta = p
    return rho*np.array([np.cos(theta), np.sin(theta)])

def gravity(p):
    return -p/sum(p**2)

In [ ]:
assert np.isclose(div(polar, np.array([1,0]), delta=1e-3), 2, rtol=1e-3)

In [ ]:
gpolar = grad(polar, np.array([1,0]), delta=1e-3)

assert np.allclose(gpolar, np.eye(2), rtol=1e-3, atol=1e-3)
assert np.sum(np.abs(gpolar - np.eye(2))) > 1e-4

In [ ]:
assert np.isclose(div(gravity, np.array([1,2,1]), delta=1e-6), -1/6, rtol=1e-6)

In [ ]:
assert np.isclose(div(gravity, np.array([1,1,1,1,1])), -0.6, rtol=1e-8)

# Parte 2: Vetorizando os vetores

Se desejarmos vetorizar a nova função `df` para que $x$ (ou $v$) possam ser "listas" (ou matrizes),
precisamos escolher uma convenção.

## Questão 6: entendendo o problema

Suponha, portanto, que temos uma lista de $K$ pontos $p_k \in R^n$.
A forma mais natural de armazenar esta lista é em uma matriz, $n \times K$ ou $K \times n$ (a ser escolhido!)
Temos, também, uma potencial lista de $L$ vetores diretores $v_l \in R^n$.
Note que a dimensão de cada $v_l$ e $p_k$ deve ser igual.
A lista de vetores vai ser ser armazenada em (outra) matriz, $n \times L$ ou $L \times n$ (na mesma ordem, certo?).

Por fim, suponha que a função $f$ vai de $R^n$ em $R^m$.
Portanto, queremos calcular (e armazenar)
$$ \frac{\partial f}{\partial (v_l)}(p_k). $$

Qual é a dimensão de cada uma destas derivadas direcionais?
Como você pensa armazenar esta estrutura de dados?

YOUR ANSWER HERE

## Questão 7: Uma convenção

Vamos **escolher** representar uma lista de $K$ vetores em $R^n$ por um `np.array` de dimensão $n \times K$.
Assim, a lista dos vetores $v_l$ também será dada da mesma forma, e a função `f` também deve ser capaz de receber
listas de vetores $n \times K$ e retornar uma lista $m \times K$.

Adapte a função `df` para receber listas de pontos / vetores diretores.
Qual a dimensão da resposta? (em função de $K$, $L$, $n$, $m$, ...)